# Structure gemini outputs with Pydantic

In [1]:
from google import genai
import pydantic

client = genai.Client()

response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "List a few asian soups recipes, a yummy description and list the ingredients",
)

print(response.text)

Oh, get ready for a delicious journey! Here are a few incredible Asian soup recipes, each with a mouth-watering description and key ingredients to get your taste buds tingling.

---

### 1. Tom Yum Goong (Spicy Thai Shrimp Soup)

**A Yummy Description:** Prepare for a flavor explosion! This vibrant Thai classic is an electrifying symphony of **spicy, sour, savory, and aromatic**. It's a clear, light broth bursting with the invigorating zing of lemongrass, galangal, and kaffir lime leaves, dancing perfectly with the plump sweetness of shrimp and earthy mushrooms. Every spoonful is a fragrant, invigorating hug that awakens all your senses!

**Key Ingredients:**
*   **Protein:** Fresh Shrimp (peeled and deveined)
*   **Aromatics:** Lemongrass (bruised), Galangal (sliced), Kaffir Lime Leaves (torn)
*   **Heat:** Thai Bird's Eye Chilies (crushed or sliced), Chili Paste in Soybean Oil (Nam Prik Pao - optional, for depth)
*   **Vegetables:** Straw Mushrooms or Oyster Mushrooms, Cherry Tomatoe

In [ ]:
response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = """List a few asian soups recipes, a yummy description and list the ingredients
        give me fields of
        recipe_name: str
        description: str
        ingredients: list[str]

        not in markdown
    """,
)

print(response.text)

recipe_name: Tom Yum Goong (Spicy Thai Shrimp Soup)
description: A vibrant and exhilarating Thai classic that perfectly balances hot, sour, and savory flavors. Each spoonful is a fragrant explosion of lemongrass, galangal, kaffir lime, and chili, beautifully complemented by plump, juicy shrimp and earthy mushrooms in a clear, zesty broth. It's a truly intoxicating and refreshing experience that awakens all your senses.
ingredients:
- Fresh shrimp
- Lemongrass stalks
- Galangal (or ginger)
- Kaffir lime leaves
- Lime juice
- Fish sauce
- Mushrooms (straw or oyster mushrooms)
- Fresh bird's eye chilies
- Cilantro
- Shallots
- Chicken or vegetable broth
- Chili paste (optional)

recipe_name: Japanese Shoyu Ramen
description: Dive into a comforting bowl of this iconic Japanese ramen, featuring springy noodles swimming in a rich, savory soy-sauce based chicken or pork broth. Each slurp is a harmonious blend of umami, topped with tender chashu pork, a perfectly jammy ajitama egg, crisp nori,

In [4]:
from pydantic import BaseModel
class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]

client=genai.Client()

response = client.models.generate_content(
    model = "gemini-2.5-flash",
    contents = "List a few asian soups recipes, a yummy description and list the ingredients",
    config = {"response_mime_type": "application/json", "response_schema": list[Recipe]}
)

print(response.text)

[{"recipe_name":"Tonkotsu Ramen","description":"A rich and creamy Japanese pork bone broth noodle soup, simmered for hours to extract maximum flavor, often topped with tender chashu pork, a soft-boiled egg, nori, and scallions. It's a deeply satisfying and comforting bowl.","ingredients":["pork bones","water","ginger","garlic","scallions","ramen noodles","chashu pork","soft-boiled eggs","nori","sesame oil","soy sauce","mirin"]}, {"recipe_name":"Tom Yum Goong","description":"A vibrant and aromatic Thai hot and sour shrimp soup, bursting with the flavors of lemongrass, galangal, kaffir lime leaves, chilies, and lime juice. It's a tantalizing balance of spicy, sour, and savory.","ingredients":["shrimp","lemongrass","galangal","kaffir lime leaves","mushrooms","fish sauce","lime juice","thai chilies","cilantro","chicken broth","evaporated milk or coconut milk (optional for creamy version)"]}, {"recipe_name":"Vietnamese Beef Pho","description":"A fragrant and savory Vietnamese noodle soup fe

In [6]:
recipes = response.parsed
recipes, len(recipes)

([Recipe(recipe_name='Tonkotsu Ramen', description="A rich and creamy Japanese pork bone broth noodle soup, simmered for hours to extract maximum flavor, often topped with tender chashu pork, a soft-boiled egg, nori, and scallions. It's a deeply satisfying and comforting bowl.", ingredients=['pork bones', 'water', 'ginger', 'garlic', 'scallions', 'ramen noodles', 'chashu pork', 'soft-boiled eggs', 'nori', 'sesame oil', 'soy sauce', 'mirin']),
  Recipe(recipe_name='Tom Yum Goong', description="A vibrant and aromatic Thai hot and sour shrimp soup, bursting with the flavors of lemongrass, galangal, kaffir lime leaves, chilies, and lime juice. It's a tantalizing balance of spicy, sour, and savory.", ingredients=['shrimp', 'lemongrass', 'galangal', 'kaffir lime leaves', 'mushrooms', 'fish sauce', 'lime juice', 'thai chilies', 'cilantro', 'chicken broth', 'evaporated milk or coconut milk (optional for creamy version)']),
  Recipe(recipe_name='Vietnamese Beef Pho', description="A fragrant and

In [7]:
recipes[0].description, recipes[0].recipe_name, recipes[0].ingredients

("A rich and creamy Japanese pork bone broth noodle soup, simmered for hours to extract maximum flavor, often topped with tender chashu pork, a soft-boiled egg, nori, and scallions. It's a deeply satisfying and comforting bowl.",
 'Tonkotsu Ramen',
 ['pork bones',
  'water',
  'ginger',
  'garlic',
  'scallions',
  'ramen noodles',
  'chashu pork',
  'soft-boiled eggs',
  'nori',
  'sesame oil',
  'soy sauce',
  'mirin'])

## Simulate house prices

In [8]:
from typing import Literal

class Home(BaseModel):
    price: int
    monthly_fee: int
    living_area: float
    number_rooms: int
    type: Literal["apartment", "house"]
    address: str



response = client.models.generate_content(
    model = "gemini-2.5-pro",
    contents = """List 50 apartments and houses in Sweden with their monthly_fee, their price, living area,
    number of rooms, address, type if it is apartment or house.
    All currencies are in SEK.""",
    config = {"response_mime_type": "application/json", "response_schema": list[Home]}
)


homes = response.parsed
homes

[Home(price=3450000, monthly_fee=2850, living_area=48.5, number_rooms=2, type='apartment', address='Hornsgatan 5A, 118 20 Stockholm'),
 Home(price=7800000, monthly_fee=4500, living_area=180.0, number_rooms=6, type='house', address='Villa Ljungvägen 12, 436 39 Askim'),
 Home(price=2950000, monthly_fee=3900, living_area=75.0, number_rooms=3, type='apartment', address='Drottningtorget 8B, 211 25 Malmö'),
 Home(price=5600000, monthly_fee=3800, living_area=155.5, number_rooms=5, type='house', address='Björkvägen 7, 756 53 Uppsala'),
 Home(price=1850000, monthly_fee=2100, living_area=38.0, number_rooms=1, type='apartment', address='Rydsvägen 250C, 584 31 Linköping'),
 Home(price=5200000, monthly_fee=5100, living_area=105.0, number_rooms=4, type='apartment', address='Sveavägen 98, 113 50 Stockholm'),
 Home(price=6300000, monthly_fee=4200, living_area=165.0, number_rooms=5, type='house', address='Änggårdsgatan 22, 413 19 Göteborg'),
 Home(price=2100000, monthly_fee=3150, living_area=62.5, numb

In [8]:
len(homes)

50

In [9]:
homes[0].price, homes[0].address

(3450000, 'Hornsgatan 5A, 118 20 Stockholm')

In [10]:
import pandas as pd

df = pd.DataFrame([home.__dict__ for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,type,address
0,3450000,2850,48.5,2,apartment,"Hornsgatan 5A, 118 20 Stockholm"
1,7800000,4500,180.0,6,house,"Villa Ljungvägen 12, 436 39 Askim"
2,2950000,3900,75.0,3,apartment,"Drottningtorget 8B, 211 25 Malmö"
3,5600000,3800,155.5,5,house,"Björkvägen 7, 756 53 Uppsala"
4,1850000,2100,38.0,1,apartment,"Rydsvägen 250C, 584 31 Linköping"


In [14]:
df.query("price < 3_000_000")

,price,monthly_fee,living_area,number_rooms,type,address
2,2950000,3900,75.0,3,apartment,"Drottningtorget 8B, 211 25 Malmö"
4,1850000,2100,38.0,1,apartment,"Rydsvägen 250C, 584 31 Linköping"
7,2100000,3150,62.5,2,apartment,"Stora Nygatan 40, 211 37 Malmö"
9,2400000,3400,80.0,3,apartment,"Klostergatan 15, 582 23 Linköping"
11,2750000,4100,78.0,3,apartment,"Linnégatan 31, 413 04 Göteborg"
13,1995000,2950,55.5,2,apartment,"Rådhusesplanaden 10, 903 28 Umeå"
14,2300000,3700,85.0,3,apartment,"Storgatan 35, 722 12 Västerås"
19,2050000,3200,70.0,3,apartment,"Drottninggatan 50, 602 32 Norrköping"
22,2500000,3850,82.0,3,apartment,"Amiralsgatan 12, 211 55 Malmö"
24,1900000,2600,45.0,2,apartment,"Järntorget 4, 703 61 Örebro"


In [15]:
cheap_houses = df.query("price < 5_000_000 and type == 'house'")
cheap_houses

,price,monthly_fee,living_area,number_rooms,type,address
8,4100000,3300,130.0,4,house,"Studentvägen 19, 752 34 Uppsala"
12,3800000,2900,115.0,4,house,"Kalkstensvägen 5, 224 78 Lund"
16,4900000,3950,145.0,5,house,"Eklandagatan 55, 412 61 Göteborg"
23,3200000,2800,125.0,4,house,"Skolgatan 8, 903 25 Umeå"
33,4800000,3700,140.0,5,house,"Luthagsesplanaden 33, 752 25 Uppsala"
36,3500000,2900,120.0,4,house,"Mariehemsvägen 7, 906 54 Umeå"
39,4500000,3400,135.0,5,house,"Stenhuggarevägen 9, 226 41 Lund"
44,3900000,3100,128.0,5,house,"Pilgatan 1, 723 48 Västerås"
47,4100000,3250,133.0,5,house,"Flogstavägen 65, 752 73 Uppsala"


In [16]:
cheap_houses.to_csv("cheap_houses.csv", index=False)